In [1]:
import openai
import json
import time
import pandas as pd
from langchain import PromptTemplate
import random
from tqdm import tqdm
from system_msg_v2 import system_prompt_difficult,system_prompt_medium

openai.api_base = "https://api.aimlapi.com/"  # use the IP or hostname of your instance
openai.api_key = '2cd60781c98843dfa09cef598719bcd1'

In [2]:
def query_qwen(query_str):
    '''
    Given a query, prompt QWEN for results
    
    query_str: str
    '''

    system_content = "You are a helpful assistant"
    client = openai.OpenAI(api_key = openai.api_key, base_url=openai.api_base)
    chat_completion = client.chat.completions.create(
        model="Qwen/Qwen1.5-72B-Chat",
        messages=[
            {"role": "system", "content": system_content},
            {"role": "user", "content": query_str},
        ],
        temperature=0.7,
        max_tokens=4096,
    )
    response = chat_completion.choices[0].message.content
    return response




In [3]:
L3_TASKS = [
    "Code synthesis",
"Code retreival",
"Text to SQL",
"Math programming",
"Code snippets & examples",
"Plot generation",
"Generating bash commands",
"Database query generation",
"UI code generation",
"Configuration file generation",
"Completing a function",
"Completing a class",
"Code infilling",
"Predicting next line of code",
"Autocomplete code blocks",
"Variable name suggestion",
"Method signature completion",
"Auto-generate test cases",
"Completing HTML tags",
"Syntax correction",
"Summarizing a file / script / repository in a paragraph (or 5 bullets)",
"Automatic commenting",
"Minify code",
"Extracting main features of code",
"Generating code abstract",
"Visual code summary",
"Code compression techniques",
"Summarizing changes in version control",
"Documentation summarization",
"Inline code summarization",
"Code modification (refactoring)",
"Code optimization",
"Code simplification",
"Code search - given a bit of code, search within it",
"API Mining - e.g. help generating calls for APIs",
"Redundancy Removal",
"Converting loops to recursion",
"Refactoring for readability",
"Refactoring for performance",
"Standardization of code format",
"Linux/Mac/Windows common CLI tasks",
"CLI package management",
"Software development environment config",
"Automated script generation",
"Environment setup automation",
"CLI shortcuts",
"CLI for cloud management",
"CLI tools for network troubleshooting",
"Command line data processing",
"Shell script optimization",
"Package management (for all languages)",
"Code repository management",
"Integration with IDEs",
"Build automation",
"Dependency resolution",
"Cross-platform compatibility checks",
"Ecosystem migration tools",
"Code sharing platforms",
"Collaborative coding tools",
"Real-time code synchronization",
"Code translation (one from language to another)",
"Cross-language API usage",
"Legacy code modernization",
"Interoperability solutions",
"Scripting to compiled code conversion",
"Automatic code localization",
"Platform-specific adaptations",
"Framework-specific code generation",
"Code porting for different OS",
"Multi-language code integration",
"Writing a javadoc for this function",
"Generating comments based on code logic",
"Automatic inline comments",
"Updating outdated comments",
"Generating comments for algorithms",
"Comment based on code complexity",
"Summarizing logical blocks with comments",
"Code annotation for review",
"Extracting and commenting critical sections",
"Tool-generated comment consistency check",
"Creating a descriptive and useful commit text for a code commit",
"Automatic commit classification",
"Semantic commit messaging",
"Code Commit message templates",
"Version control integration for commit messages",
"Multi-language commit support",
"Code Commit summarization for changelogs",
"Context-aware commit suggestions",
"Feature-specific commit messages",
"Code Commit message consistency checker",
"Writing a docstring",
"Extended documentation with examples",
"API endpoint documentation",
"Function parameter details",
"Error handling documentation",
"Performance notes",
"Usage scenarios",
"Deprecation notices",
"Security implications",
"Compatibility notes",
"Basic usage scenario",
"Advanced usage scenario",
"Performance critical use case",
"Error handling example",
"Integration with other functions",
"Cross-platform usage example",
"Thread safety demonstration",
"Usage with optional parameters",
"Deprecation alternatives",
"Common pitfalls and workarounds",
"Endpoint description",
"Parameter details",
"Return values",
"Authentication requirements",
"Error codes explanation",
"Sample request/response",
"Versioning and compatibility",
"Deprecation policy",
"Rate limits",
"Security guidelines",
"Code repair",
"Bug identification",
"Code fix suggestions",
"Defect detection",
"Clone detection",
"bugs in a code",
"fixing the bugs in a code",
" an error message/traceback",
"fixing an error message <message> in a code",
"Debugging Strategies/Tooling",
"Unit Test Generation",
"Testing Strategy (e.g. frameworks/guides)",
"Automated regression testing",
"Integration testing tools",
"Continuous testing practices",
"Load and performance testing",
"Security penetration testing",
"User acceptance testing",
"Code coverage analysis",
"Test data generation",
"Deobfuscation",
"Code classification",
"Peer review automation",
"Static code analysis",
"Code style enforcement",
"Security review integration",
"Review metrics and dashboards",
"Automated refactor suggestions",
"Code smell detection",
"Best practices checklist",
"Identifying mistakes that allow for XSS injection",
"SQL injection prevention",
"Code audit for security vulnerabilities",
"Encryption standards review",
"Authentication mechanism review",
"Access control checks",
"Data privacy compliance",
"Security best practices",
"Third-party library security",
"Secure coding training",
"Code fuzzing",
"Assertion Generation",
"Automated code quality reports",
"Performance profiling",
"Memory leak detection",
"Usability testing",
"Cross-browser testing",
"Mobile responsiveness testing",
"Accessibility compliance",
"Internationalization checks",
"Parsing logs into structured templates",
"Finding anomalies from raw logs",
"Log event correlation",
"Predictive log analysis",
"Log-based alerting",
"Real-time log monitoring",
"Log archiving strategies",
"Log data visualization",
"User behavior analysis from logs",
"Security incident detection through logs"
]

In [4]:
from attributes_batch3 import attributes_dict
all_attributes=attributes_dict['Topics']
len(all_attributes)

3

In [5]:
programming_language_concepts_template="""
Generate {NoQ} questions and their responses with example code in {LANGUAGE} language to identify and explain {L3_TASK} to understand {PROGRAMMING_CONCEPT_CONTENT} in {CATEGORY}. The level of complexity of each questions MUST BE extremely difficult that can truly test the capability of experts from this field.
"""
programming_language_concepts_prompt=PromptTemplate.from_template(programming_language_concepts_template)

focus_framework_language_template="""
Generate {NoQ} questions and their responses with example code in {LANGUAGE} language to identify and explain {L3_TASK} in the context of {FOCUS} with {FRAMEWORK} framework. The level of complexity of each questions MUST BE extremely difficult that can truly test the capability of experts from this field.
"""
focus_framework_language_prompt=PromptTemplate.from_template(focus_framework_language_template)

focus_language_template="""
Generate {NoQ} questions and their responses with example code in {LANGUAGE} language to identify and explain {L3_TASK} in the context of {FOCUS}. The level of complexity of each questions MUST BE extremely difficult that can truly test the capability of experts from this field.
"""
focus_language_prompt=PromptTemplate.from_template(focus_language_template)

focus_language_technology_template="""
Generate {NoQ} questions and their responses with example code in {LANGUAGE} language to identify and explain {L3_TASK} in the context of {TECHNOLOGY} for {FOCUS}. The level of complexity of each questions MUST BE extremely difficult that can truly test the capability of experts from this field.
"""
focus_language_technology_prompt=PromptTemplate.from_template(focus_language_technology_template)


In [6]:
per_concept_ques=3
def generate_input_prompts(L3_TASKS,all_attributes,programming_language_concepts_prompt,focus_framework_language_prompt,focus_language_prompt,focus_language_technology_prompt,per_concept_ques):
    input_prompts=[]
    for l3_task in tqdm(L3_TASKS):
        t1=time.time()
        for i in range(len(all_attributes)):
            element=all_attributes[i]
            if element['Category']=="Programming Language Concepts" or element['Category']=="DS/Algo":
                category=element['Category']
                subcategories=element['Subcategories']
                languages=element['languages']
                for language in languages:
                    for item in subcategories:
                        for content in item['Contents']:
                            input_prompt=programming_language_concepts_prompt.format_prompt(NoQ=per_concept_ques,LANGUAGE=language,L3_TASK=l3_task,PROGRAMMING_CONCEPT_CONTENT=content,CATEGORY=category).to_string()
                            input_prompts.append(input_prompt)
                            print('input_prompt for programming concept and DS Algo:',input_prompt)
                            print('#'*100)
            elif element['Category']=="Development":
                subcategories=element['Subcategories']
                for subcategory in subcategories:
                    if subcategory['Topic']=="Web Development":
                        subtopics=subcategory["Subtopics"]
                        for subtopic in subtopics:
                            focus=subtopic["Focus"]
                            languages=subtopic["languages"]
                            if "Frameworks" in subtopic.keys():
                                frameworks=subtopic["Frameworks"]
                                for framework in frameworks:
                                    for language in languages:
                                        input_prompt=focus_framework_language_prompt.format_prompt(NoQ=per_concept_ques,LANGUAGE=language,L3_TASK=l3_task,FOCUS=focus,FRAMEWORK=framework).to_string()
                                        input_prompts.append(input_prompt)
                                        print('input_prompt for focus_framework_language in web development:',input_prompt)
                                        print('#'*100)
                            else:
                                for language in languages:
                                    input_prompt=focus_language_prompt.format_prompt(NoQ=per_concept_ques,LANGUAGE=language,L3_TASK=l3_task,FOCUS=focus).to_string()
                                    input_prompts.append(input_prompt)
                                    print('input_prompt for focus_language in web development :',input_prompt)
                                    print('#'*100)
                    elif subcategory['Topic']=="Mobile Development":
                        subtopics=subcategory["Subtopics"]
                        for subtopic in subtopics:
                            focus=subtopic["Focus"]
                            languages=subtopic["languages"]
                            for language in languages:
                                input_prompt=focus_language_prompt.format_prompt(NoQ=per_concept_ques,LANGUAGE=language,L3_TASK=l3_task,FOCUS=focus).to_string()
                                input_prompts.append(input_prompt)
                                print('input_prompt for focus_language in mobile development:',input_prompt)
                                print('#'*100)
                    elif subcategory['Topic']=="Software Development":
                        subtopics=subcategory["Subtopics"]
                        for subtopic in subtopics:
                            focus=subtopic["Focus"]
                            languages=subtopic["languages"]
                            for language in languages:
                                input_prompt=focus_language_prompt.format_prompt(NoQ=per_concept_ques,LANGUAGE=language,L3_TASK=l3_task,FOCUS=focus).to_string()
                                input_prompts.append(input_prompt)
                                print('input_prompt for focus_language in Software Development:',input_prompt)
                                print('#'*100)
                    elif subcategory['Topic']=="Data Science":
                        subtopics=subcategory["Subtopics"]
                        for subtopic in subtopics:
                            focus=subtopic["Focus"]
                            languages=subtopic["languages"]
                            if "Technologies" in subtopic.keys():
                                technologies=subtopic["Technologies"]
                                for technology in technologies:
                                    for language in languages:
                                        input_prompt=focus_language_technology_prompt.format_prompt(NoQ=per_concept_ques,LANGUAGE=language,L3_TASK=l3_task,FOCUS=focus,TECHNOLOGY=technology).to_string()
                                        input_prompts.append(input_prompt)
                                        print('input_prompt for focus_technology_language in data science:',input_prompt)
                                        print('#'*100)
                            else:
                                for language in languages:
                                    input_prompt=focus_language_prompt.format_prompt(NoQ=per_concept_ques,LANGUAGE=language,L3_TASK=l3_task,FOCUS=focus).to_string()
                                    input_prompts.append(input_prompt)
                                    print('input_prompt for focus_language in data science :',input_prompt)
                                    print('#'*100)
                    elif subcategory['Topic']=="Systems Programming":
                        subtopics=subcategory["Subtopics"]
                        for subtopic in subtopics:
                            focus=subtopic["Focus"]
                            languages=subtopic["languages"]
                            if "Technologies" in subtopic.keys():
                                technologies=subtopic["Technologies"]
                                for technology in technologies:
                                    for language in languages:
                                        input_prompt=focus_language_technology_prompt.format_prompt(NoQ=per_concept_ques,LANGUAGE=language,L3_TASK=l3_task,FOCUS=focus,TECHNOLOGY=technology).to_string()
                                        input_prompts.append(input_prompt)
                                        print('input_prompt for focus_technology_language in Systems Programming:',input_prompt)
                                        print('#'*100)
                    elif subcategory['Topic']=="Database Management":
                        subtopics=subcategory["Subtopics"]
                        for subtopic in subtopics:
                            focus=subtopic["Focus"]
                            languages=subtopic["languages"]
                            if "Technologies" in subtopic.keys():
                                technologies=subtopic["Technologies"]
                                for technology in technologies:
                                    for language in languages:
                                        input_prompt=focus_language_technology_prompt.format_prompt(NoQ=per_concept_ques,LANGUAGE=language,L3_TASK=l3_task,FOCUS=focus,TECHNOLOGY=technology).to_string()
                                        input_prompts.append(input_prompt)
                                        print('input_prompt for focus_technology_language in Database Management:',input_prompt)
                                        print('#'*100)
                    elif subcategory['Topic']=="Cloud Computing":
                        subtopics=subcategory["Subtopics"]
                        for subtopic in subtopics:
                            focus=subtopic["Focus"]
                            languages=subtopic["languages"]
                            if "Technologies" in subtopic.keys():
                                technologies=subtopic["Technologies"]
                                for technology in technologies:
                                    for language in languages:
                                        input_prompt=focus_language_technology_prompt.format_prompt(NoQ=per_concept_ques,LANGUAGE=language,L3_TASK=l3_task,FOCUS=focus,TECHNOLOGY=technology).to_string()
                                        input_prompts.append(input_prompt)
                                        print('input_prompt for focus_technology_language in Cloud Computing:',input_prompt)
                                        print('#'*100)
        
    with open("./input/input_prompts.json",'w') as f:
        json.dump({"input_prompts":input_prompts},f)

generate_input_prompts(L3_TASKS,all_attributes,programming_language_concepts_prompt,focus_framework_language_prompt,focus_language_prompt,focus_language_technology_prompt,per_concept_ques)

  0%|          | 0/180 [00:00<?, ?it/s]


KeyError: 'languages'

In [7]:
programming_language_concepts_template="""
Generate a question and its response with example code in {LANGUAGE} language to identify and explain {L3_TASK} to understand {PROGRAMMING_CONCEPT_CONTENT} in {CATEGORY}. The level of complexity of the question MUST BE extremely difficult that can truly test the capability of experts from this field.
"""
programming_language_concepts_prompt=PromptTemplate.from_template(programming_language_concepts_template)

focus_framework_language_template="""
Generate a question and its response with example code in {LANGUAGE} language to identify and explain {L3_TASK} in the context of {FOCUS} with {FRAMEWORK} framework. The level of complexity of the question MUST BE extremely difficult that can truly test the capability of experts from this field.
"""
focus_framework_language_prompt=PromptTemplate.from_template(focus_framework_language_template)

focus_language_template="""
Generate a question and its response with example code in {LANGUAGE} language to identify and explain {L3_TASK} in the context of {FOCUS}. The level of complexity of the question MUST BE extremely difficult that can truly test the capability of experts from this field.
"""
focus_language_prompt=PromptTemplate.from_template(focus_language_template)

focus_language_technology_template="""
Generate a question and its response with example code in {LANGUAGE} language to identify and explain {L3_TASK} in the context of {TECHNOLOGY} for {FOCUS}. The level of complexity of the question MUST BE extremely difficult that can truly test the capability of experts from this field.
"""
focus_language_technology_prompt=PromptTemplate.from_template(focus_language_technology_template)

In [8]:
def generate_input_prompts_batch3(L3_TASKS,all_attributes,programming_language_concepts_prompt,focus_framework_language_prompt,focus_language_prompt,focus_language_technology_prompt):
    input_prompts=[]
    for l3_task in tqdm(L3_TASKS):
        t1=time.time()
        for i in range(len(all_attributes)):
            element=all_attributes[i]
            if element['Category']=="DS/Algo":
                category=element['Category']
                subcategories=element['Subcategories']
                languages=element['languages']
                for language in languages:
                    for item in subcategories:
                        for content in item['Contents']:
                            input_prompt=programming_language_concepts_prompt.format_prompt(LANGUAGE=language,L3_TASK=l3_task,PROGRAMMING_CONCEPT_CONTENT=content,CATEGORY=category).to_string()
                            input_prompts.append(input_prompt)
                            print('input_prompt for programming concept and DS Algo:',input_prompt)
                            print('#'*100)
            elif element['Category']=="Programming Language Concepts":
                category=element['Category']
                subcategories=element['Subcategories']
                for item in subcategories:
                    languages=item['languages']
                    for language in languages:
                        for content in item['Contents']:
                            input_prompt=programming_language_concepts_prompt.format_prompt(LANGUAGE=language,L3_TASK=l3_task,PROGRAMMING_CONCEPT_CONTENT=content,CATEGORY=category).to_string()
                            input_prompts.append(input_prompt)
                            print('input_prompt for programming concept and DS Algo:',input_prompt)
                            print('#'*100)
            elif element['Category']=="Development":
                subcategories=element['Subcategories']
                for subcategory in subcategories:
                    if subcategory['Topic']=="Web Development":
                        subtopics=subcategory["Subtopics"]
                        for subtopic in subtopics:
                            focus=subtopic["Focus"]
                            languages=subtopic["languages"]
                            if "Frameworks" in subtopic.keys():
                                frameworks=subtopic["Frameworks"]
                                for framework in frameworks:
                                    for language in languages:
                                        input_prompt=focus_framework_language_prompt.format_prompt(LANGUAGE=language,L3_TASK=l3_task,FOCUS=focus,FRAMEWORK=framework).to_string()
                                        input_prompts.append(input_prompt)
                                        print('input_prompt for focus_framework_language in web development:',input_prompt)
                                        print('#'*100)
                            else:
                                for language in languages:
                                    input_prompt=focus_language_prompt.format_prompt(LANGUAGE=language,L3_TASK=l3_task,FOCUS=focus).to_string()
                                    input_prompts.append(input_prompt)
                                    print('input_prompt for focus_language in web development :',input_prompt)
                                    print('#'*100)
                    elif subcategory['Topic']=="Mobile Development":
                        subtopics=subcategory["Subtopics"]
                        for subtopic in subtopics:
                            focus=subtopic["Focus"]
                            languages=subtopic["languages"]
                            for language in languages:
                                input_prompt=focus_language_prompt.format_prompt(LANGUAGE=language,L3_TASK=l3_task,FOCUS=focus).to_string()
                                input_prompts.append(input_prompt)
                                print('input_prompt for focus_language in mobile development:',input_prompt)
                                print('#'*100)
                    elif subcategory['Topic']=="Software Development":
                        subtopics=subcategory["Subtopics"]
                        for subtopic in subtopics:
                            focus=subtopic["Focus"]
                            languages=subtopic["languages"]
                            for language in languages:
                                input_prompt=focus_language_prompt.format_prompt(LANGUAGE=language,L3_TASK=l3_task,FOCUS=focus).to_string()
                                input_prompts.append(input_prompt)
                                print('input_prompt for focus_language in Software Development:',input_prompt)
                                print('#'*100)
                    elif subcategory['Topic']=="Data Science":
                        subtopics=subcategory["Subtopics"]
                        for subtopic in subtopics:
                            focus=subtopic["Focus"]
                            languages=subtopic["languages"]
                            if "Technologies" in subtopic.keys():
                                technologies=subtopic["Technologies"]
                                for technology in technologies:
                                    for language in languages:
                                        input_prompt=focus_language_technology_prompt.format_prompt(LANGUAGE=language,L3_TASK=l3_task,FOCUS=focus,TECHNOLOGY=technology).to_string()
                                        input_prompts.append(input_prompt)
                                        print('input_prompt for focus_technology_language in data science:',input_prompt)
                                        print('#'*100)
                            else:
                                for language in languages:
                                    input_prompt=focus_language_prompt.format_prompt(LANGUAGE=language,L3_TASK=l3_task,FOCUS=focus).to_string()
                                    input_prompts.append(input_prompt)
                                    print('input_prompt for focus_language in data science :',input_prompt)
                                    print('#'*100)
                    elif subcategory['Topic']=="Systems Programming":
                        subtopics=subcategory["Subtopics"]
                        for subtopic in subtopics:
                            focus=subtopic["Focus"]
                            languages=subtopic["languages"]
                            if "Technologies" in subtopic.keys():
                                technologies=subtopic["Technologies"]
                                for technology in technologies:
                                    for language in languages:
                                        input_prompt=focus_language_technology_prompt.format_prompt(LANGUAGE=language,L3_TASK=l3_task,FOCUS=focus,TECHNOLOGY=technology).to_string()
                                        input_prompts.append(input_prompt)
                                        print('input_prompt for focus_technology_language in Systems Programming:',input_prompt)
                                        print('#'*100)
                    elif subcategory['Topic']=="Database Management":
                        subtopics=subcategory["Subtopics"]
                        for subtopic in subtopics:
                            focus=subtopic["Focus"]
                            languages=subtopic["languages"]
                            if "Technologies" in subtopic.keys():
                                technologies=subtopic["Technologies"]
                                for technology in technologies:
                                    for language in languages:
                                        input_prompt=focus_language_technology_prompt.format_prompt(LANGUAGE=language,L3_TASK=l3_task,FOCUS=focus,TECHNOLOGY=technology).to_string()
                                        input_prompts.append(input_prompt)
                                        print('input_prompt for focus_technology_language in Database Management:',input_prompt)
                                        print('#'*100)
                    elif subcategory['Topic']=="Cloud Computing":
                        subtopics=subcategory["Subtopics"]
                        for subtopic in subtopics:
                            focus=subtopic["Focus"]
                            languages=subtopic["languages"]
                            if "Technologies" in subtopic.keys():
                                technologies=subtopic["Technologies"]
                                for technology in technologies:
                                    for language in languages:
                                        input_prompt=focus_language_technology_prompt.format_prompt(LANGUAGE=language,L3_TASK=l3_task,FOCUS=focus,TECHNOLOGY=technology).to_string()
                                        input_prompts.append(input_prompt)
                                        print('input_prompt for focus_technology_language in Cloud Computing:',input_prompt)
                                        print('#'*100)
        
    with open("./input/input_prompts_batch3.json",'w') as f:
        json.dump({"input_prompts":input_prompts},f)

generate_input_prompts_batch3(L3_TASKS,all_attributes,programming_language_concepts_prompt,focus_framework_language_prompt,focus_language_prompt,focus_language_technology_prompt)

 47%|████▋     | 85/180 [00:00<00:00, 419.76it/s]IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

 92%|█████████▏| 165/180 [00:00<00:00, 369.96it/s]IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

100%|██████████| 180/180 [00:00<00:00, 363.57it/s]



####################################################################################################
input_prompt for focus_language in data science : 
Generate a question and its response with example code in R language to identify and explain Mobile responsiveness testing in the context of Data Analysis. The level of complexity of the question MUST BE extremely difficult that can truly test the capability of experts from this field.

####################################################################################################
input_prompt for focus_language in data science : 
Generate a question and its response with example code in SQL language to identify and explain Mobile responsiveness testing in the context of Data Analysis. The level of complexity of the question MUST BE extremely difficult that can truly test the capability of experts from this field.

####################################################################################################
input_prompt for

In [9]:
L3_Task,level,language,category,topic,contents,llm-response
Batch3_multi_language_concept

NameError: name 'L3_Task' is not defined

### Using System Prompt

In [10]:
def query_qwen(query_str,system_content):
    '''
    Given a query, prompt QWEN for results
    
    query_str: str
    '''

    #system_content = "You are a helpful assistant"
    client = openai.OpenAI(api_key = openai.api_key, base_url=openai.api_base)
    chat_completion = client.chat.completions.create(
        model="Qwen/Qwen1.5-72B-Chat",
        messages=[
            {"role": "system", "content": system_content},
            {"role": "user", "content": query_str},
        ],
        temperature=0.7,
        max_tokens=4096,
    )
    response = chat_completion.choices[0].message.content
    return response


In [3]:
prompt_template="""Generate {NO_QUES} questions and their responses with example code in suitable language for Area: “{L2_TASK}”, Sub Area: “{L3_TASK}”. The level of complexity has to be difficult. """
prompt=PromptTemplate.from_template(prompt_template)

def generate_code_prompts(input_file,output_file,prompt,system_prompt_txt,per_topic_ques):
    df=pd.read_excel(input_file)
    #print(df.head)
    df_dict_list=df.to_dict('records')
    out_dict_list=[]
    for i in tqdm(range(len(df_dict_list))):
        t1=time.time()
        l2_task=df_dict_list[i]['L2_Categories']
        l3_task=df_dict_list[i]['L3_Categories']
        input_prompt = prompt.format_prompt(L2_TASK=l2_task,L3_TASK=l3_task,NO_QUES=per_topic_ques).to_string()
        response=query_qwen(input_prompt,system_prompt_txt)
        print('\n input prompt :\n',input_prompt)
        print('\n response:\n',response)
        t2=time.time()
        print("\n Time taken in response generation :",t2-t1)
        print('-'*100)
        out_dict_list.append({"L2_TASK":l2_task,"L3_TASK":l3_task,"Response":response})
        with open(output_file, "w") as f:
            json.dump(out_dict_list, f)
        
input_file="./input/categories_L1L2L3_Sheet3.xlsx"
output_file="./output/Batch3_multi_language_concepts_language_extraction.json"
per_topic_ques=2
generate_code_prompts(input_file,output_file,prompt,system_prompt_txt,per_topic_ques)

NameError: name 'system_prompt_txt' is not defined

In [4]:
levels=["medium","extremely difficult"]
prompt_template="""Generate {NO_QUES} questions and their responses with example code in suitable language for Area: “{L2_TASK}”, Sub Area: “{L3_TASK}”. The level of complexity has to be {LEVEL}."""
prompt=PromptTemplate.from_template(prompt_template)

def generate_code_prompts(input_file,output_file,prompt,system_prompt_difficult,system_prompt_medium,levels,per_topic_ques):
    df=pd.read_excel(input_file)
    #print(df.head)
    df_dict_list=df.to_dict('records')
    out_dict_list=[]
    for i in tqdm(range(len(df_dict_list))):
        t1=time.time()
        l2_task=df_dict_list[i]['L2 Category']
        l3_task=df_dict_list[i]['L3 Category']
        for level in levels:
            input_prompt = prompt.format_prompt(L2_TASK=l2_task,L3_TASK=l3_task,NO_QUES=per_topic_ques,LEVEL=level).to_string()
            if level=="medium":
                response=query_qwen(input_prompt,system_prompt_medium)
            else:
                response=query_qwen(input_prompt,system_prompt_difficult)
            print('\n input prompt :\n',input_prompt)
            print('\n response:\n',response)
            print('-'*100)
            out_dict_list.append({"L2_TASK":l2_task,"L3_TASK":l3_task,"Complexity_Level":level,"Response":response})
            with open(output_file, "w") as f:
                json.dump(out_dict_list, f)
        t2=time.time()
        print("\n Time taken in response generation :",t2-t1)
        assert False
        
        
input_file="./input/L2_L3_Tasks.xlsx"
output_file="./output/Batch3_multi_language_concepts_v1.json"
per_topic_ques=2
generate_code_prompts(input_file,output_file,prompt,system_prompt_txt,per_topic_ques)

  0%|          | 0/180 [00:00<?, ?it/s]


 input prompt :
 Generate 2 questions and their responses with example code in suitable language for Area: “Code generation (e.g., Text to Code)”, Sub Area: “Code synthesis”. The level of complexity has to be medium.

 response:
 Area: "Code generation (e.g., Text to Code)"
Sub Area: "Code synthesis"

Problem 1: Generate a Python function that takes a list of numbers and returns the sum of all even numbers in the list.
Concept: Code synthesis, Python programming

Solution:

```python
def sum_even_numbers(numbers):
    total = 0
    for num in numbers:
        if num % 2 == 0:
            total += num
    return total

input_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
result = sum_even_numbers(input_list)
print(f"The sum of even numbers in the list is: {result}")
```

Problem 2: Create a JavaScript function that receives a string and returns the string in reverse, but with all vowels capitalized.
Concept: Code synthesis, JavaScript programming

Solution:

```javascript
function reverseCapit

  0%|          | 0/180 [00:27<?, ?it/s]


 input prompt :
 Generate 2 questions and their responses with example code in suitable language for Area: “Code generation (e.g., Text to Code)”, Sub Area: “Code synthesis”. The level of complexity has to be extremely difficult.

 response:
 "Code generation (e.g., Text to Code)" is an emerging area that falls outside of the provided JSON domains, which focuses on traditional programming language concepts, DS/Algo, development, and other related topics. However, I can create a hypothetical problem that combines elements of "Programming Language Concepts" and "DS/Algo" to illustrate a challenging task related to code synthesis.

Problem:
Given a natural language description of a complex algorithm, synthesize the corresponding code in a given programming language (e.g., Python). The algorithm should take a directed graph, represented as an adjacency matrix, and return the minimum spanning tree using Kruskal's algorithm while also handling cycles and ensuring time complexity of O(E log 

AssertionError: 

#### Merging multiple json files into a single one

In [13]:
import os
output_folder="./output/Batch3/iter_0_9"
num_iteration=10
final_output_path="./output/Batch3/Batch3_all_languages_concept.json"
all_data=[]
for i in tqdm(range(num_iteration)):
    output_filename="Batch3_all_languages_concept_iter_{}.json".format(str(i))
    output_path=os.path.join(output_folder,output_filename)
    with open(output_path,'r') as f:
        data=json.load(f)
        all_data=all_data + data
len(all_data)
with open(final_output_path,'w') as f1:
    json.dump(all_data, f1)

100%|██████████| 10/10 [00:00<00:00, 182.68it/s]


#### Testing Langchain Output Parser

In [3]:
from langchain.output_parsers import StructuredOutputParser,ResponseSchema
response_schemas = [
    ResponseSchema(name="Question", description="Question generated by the model based on the instruction"),
    ResponseSchema(name="Solution", description="Solution generated by the model for its generated question"),
    ResponseSchema(name="Concept", description="Concept mentioned in generated the question"),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

levels=["medium","extremely difficult"]
prompt_template="""Generate a question and its response with example code in suitable language for Area: “{L2_TASK}”, Sub Area: “{L3_TASK}”. The level of complexity has to be {LEVEL}.
{format_instructions}
"""
prompt=PromptTemplate(
    template=prompt_template,
    input_variables=["L2_TASK","L3_TASK","LEVEL","format_instructions"],
    output_parser=output_parser
    )

def generate_code_prompts(input_file,output_file,prompt,system_prompt_difficult,system_prompt_medium,levels):
    df=pd.read_excel(input_file)
    #print(df.head)
    df_dict_list=df.to_dict('records')
    out_dict_list=[]
    for i in tqdm(range(10)):
        t1=time.time()
        l2_task=df_dict_list[i]['L2 Category']
        l3_task=df_dict_list[i]['L3 Category']
        for level in levels:
            input_prompt = prompt.format_prompt(L2_TASK=l2_task,L3_TASK=l3_task,LEVEL=level,format_instructions=format_instructions).to_string()
            if level=="medium":
                response=query_qwen(input_prompt,system_prompt_medium)
            else:
                response=query_qwen(input_prompt,system_prompt_difficult)
            print('\n input prompt :\n',input_prompt)
            print('\n response:\n',response)
            #response = response.translate(str.maketrans('', '', ''.join(map(chr, range(32)))))
            #print('\n  sanitized response:\n',response)
            try:
                parsed_response=output_parser.parse(response)
                print('\n response:\n',parsed_response)
                print('-'*100)
                out_dict_list.append({"L2_TASK":l2_task,"L3_TASK":l3_task,"Complexity_Level":level,"Question":parsed_response['Question'],"Solution":parsed_response['Solution'],"Concept":parsed_response["Concept"]})
                with open(output_file, "w") as f:
                    json.dump(out_dict_list, f)
            except Exception as e: print(e)
                
        t2=time.time()
        print("\n Time taken in response generation :",t2-t1)
        
        
        
input_file="./input/L2_L3_Tasks.xlsx"
output_file="./output/Batch4/Batch4_multi_language_concepts_v1.json"
per_topic_ques=2
generate_code_prompts(input_file,output_file,prompt,system_prompt_difficult,system_prompt_medium,levels)

  0%|          | 0/10 [00:00<?, ?it/s]


 input prompt :
 Generate a question and its response with example code in suitable language for Area: “Code generation (e.g., Text to Code)”, Sub Area: “Code synthesis”. The level of complexity has to be medium.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"Question": string  // Question generated by the model based on the instruction
	"Solution": string  // Solution generated by the model for its generated question
	"Concept": string  // Concept mentioned in generated the question
}
```


 response:
 ```json
{
  "Question": "Given a set of user-defined functions and input data, generate Python code that applies these functions in a specific order to the input.",
  "Solution": "```py
def square(x):
    return x ** 2

def multiply_by_3(x):
    return 3 * x

def apply_functions(data, functions):
    result = data
    for func in functions:
        if func == 'square':
            resu

 10%|█         | 1/10 [00:20<03:06, 20.70s/it]


 input prompt :
 Generate a question and its response with example code in suitable language for Area: “Code generation (e.g., Text to Code)”, Sub Area: “Code synthesis”. The level of complexity has to be extremely difficult.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"Question": string  // Question generated by the model based on the instruction
	"Solution": string  // Solution generated by the model for its generated question
	"Concept": string  // Concept mentioned in generated the question
}
```


 response:
 ```json
{
  "Question": "Given a complex natural language description of a web application involving user authentication, database interactions, and real-time updates, synthesize the code for a server-side API in Node.js using Express, with authentication handled by JWT, database connection with Postgres, and real-time updates via WebSockets.",
  "Solution": "```javascript

 20%|██        | 2/10 [00:36<02:22, 17.76s/it]

...


 input prompt :
 Generate a question and its response with example code in suitable language for Area: “Code generation (e.g., Text to Code)”, Sub Area: “Database query generation”. The level of complexity has to be extremely difficult.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"Question": string  // Question generated by the model based on the instruction
	"Solution": string  // Solution generated by the model for its generated question
	"Concept": string  // Concept mentioned in generated the question
}
```


 response:
 ```json
{
	"Question": "Given a complex JSON document representing a database of sales transactions with nested data structures, generate a SQL query that retrieves the top 5 customers by total purchase amount, excluding any transactions with a status of 'canceled', 'refunded', or 'pending'. The query should also consider different currencies and convert them to

 90%|█████████ | 9/10 [02:28<00:16, 16.26s/it]


 input prompt :
 Generate a question and its response with example code in suitable language for Area: “Code generation (e.g., Text to Code)”, Sub Area: “UI code generation”. The level of complexity has to be extremely difficult.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"Question": string  // Question generated by the model based on the instruction
	"Solution": string  // Solution generated by the model for its generated question
	"Concept": string  // Concept mentioned in generated the question
}
```


 response:
 ```json
{
	"Question": "Given a comprehensive design specification in Figma, develop a dynamic UI code generator that can output both React and Angular code with proper state management, responsive layout, and custom hooks for efficient performance. The generator should also include an option to generate code for server-side rendering.",
	"Solution": "Unfortunately, a 

100%|██████████| 10/10 [02:52<00:00, 17.21s/it]


 input prompt :
 Generate a question and its response with example code in suitable language for Area: “Code generation (e.g., Text to Code)”, Sub Area: “Configuration file generation”. The level of complexity has to be extremely difficult.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"Question": string  // Question generated by the model based on the instruction
	"Solution": string  // Solution generated by the model for its generated question
	"Concept": string  // Concept mentioned in generated the question
}
```


 response:
 ```json
{
	"Question": "Given a complex JSON schema representing a microservices architecture with interdependent configurations and dynamic environment variables, generate a robust, version-controlled, and portable TOML configuration file for automating the deployment of these microservices on Kubernetes.",
	"Solution": "```toml
[metadata]
  version = "1.2.

#### Output with NO_QUES=2
 input prompt :
 Generate 2 questions and their responses with example code in suitable language for Area: “Code generation (e.g., Text to Code)”, Sub Area: “Code synthesis”. The level of complexity has to be medium.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"Question": string  // Question generated by the model based on the instruction
	"Solution": string  // Solution generated by the model for its generated question
	"Concept": string  // Concept mentioned in generated the question
}
```


 response:
 ```json
[
  {
    "Question": "Given a list of ingredient dictionaries, generate a Python function that takes the list and outputs a formatted recipe string. Each ingredient should be listed with its quantity and unit.",
    "Solution": "```python
def format_recipe(ingredients):
    recipe = ''
    for ingredient in ingredients:
        if 'quantity' in ingredient and 'unit' in ingredient and 'name' in ingredient:
            recipe += f'{ingredient["quantity"]}{ingredient["unit"]} {ingredient["name"]}, '
    return recipe[:-2]  # Remove the trailing comma and space

ingredients = [
    {"quantity": 2, "unit": "cups", "name": "flour"},
    {"quantity": 1, "unit": "teaspoon", "name": "baking powder"},
    {"quantity": 1/2, "unit": "cup", "name": "butter", "temperature": "room temperature"},
    {"quantity": 3, "unit": "large", "name": "eggs"}
]

formatted_recipe = format_recipe(ingredients)
print(formatted_recipe)
```
",
    "Concept": "Code synthesis from a list of dictionaries"
  },
  {
    "Question": "Given a JSON object representing a user profile, write a JavaScript function to convert it into a properly formatted Markdown profile summary.",
    "Solution": "```javascript
function userProfileToMarkdown(user) {
    let markdown = `# ${user.name}\n\n`;
    
    if (user.title) markdown += `**Title:** ${user.title}\n`;
    if (user.experience) markdown += `**Experience:** ${user.experience} years\n`;
    if (user.skills && user.skills.length > 0) {
        markdown += `**Skills:**\n`;
        user.skills.forEach(skill => markdown += `- ${skill}\n`);
    }
    if (user.email) markdown += `**Email:** ${user.email}\n`;
    if (user.website) markdown += `**Website:** [${user.website}](${user.website})\n`;
    
    return markdown;
}

const user = {
    name: "Jane Doe",
    title: "Software Engineer",
    experience: 5,
    skills: ["JavaScript", "React", "Node.js"],
    email: "jane.doe@example.com",
    website: "https://janedoe.com"
};

console.log(userProfileToMarkdown(user));
```
",
    "Concept": "Code synthesis from JSON to Markdown"
  }
]
```

---------------------------------------------------------------------------
JSONDecodeError                           Traceback (most recent call last)
File ~/.local/lib/python3.10/site-packages/langchain/output_parsers/json.py:163, in parse_and_check_json_markdown(text, expected_keys)
    162 try:
--> 163     json_obj = parse_json_markdown(text)
    164 except json.JSONDecodeError as e:

File ~/.local/lib/python3.10/site-packages/langchain/output_parsers/json.py:145, in parse_json_markdown(json_string, parser)
    144 # Parse the JSON string into a Python dictionary
--> 145 parsed = parser(json_str)
    147 return parsed

File /home/shared/.miniconda3/envs/py10cuda117/lib/python3.10/json/__init__.py:346, in loads(s, cls, object_hook, parse_float, parse_int, parse_constant, object_pairs_hook, **kw)
    343 if (cls is None and object_hook is None and
    344         parse_int is None and parse_float is None and
    345         parse_constant is None and object_pairs_hook is None and not kw):
--> 346     return _default_decoder.decode(s)
    347 if cls is None:

File /home/shared/.miniconda3/envs/py10cuda117/lib/python3.10/json/decoder.py:337, in JSONDecoder.decode(self, s, _w)
    333 """Return the Python representation of ``s`` (a ``str`` instance
    334 containing a JSON document).
    335 
    336 """
--> 337 obj, end = self.raw_decode(s, idx=_w(s, 0).end())
    338 end = _w(s, end).end()

File /home/shared/.miniconda3/envs/py10cuda117/lib/python3.10/json/decoder.py:353, in JSONDecoder.raw_decode(self, s, idx)
    352 try:
--> 353     obj, end = self.scan_once(s, idx)
    354 except StopIteration as err:

JSONDecodeError: Invalid control character at: line 4 column 27 (char 239)

During handling of the above exception, another exception occurred:

OutputParserException                     Traceback (most recent call last)
Cell In[21], line 51
     49 output_file="./output/Batch3_multi_language_concepts_v1.json"
     50 per_topic_ques=2
---> 51 generate_code_prompts(input_file,output_file,prompt,system_prompt_difficult,system_prompt_medium,levels,per_topic_ques)

Cell In[21], line 37, in generate_code_prompts(input_file, output_file, prompt, system_prompt_difficult, system_prompt_medium, levels, per_topic_ques)
     35 print('\n input prompt :\n',input_prompt)
     36 print('\n response:\n',response)
---> 37 parsed_response=output_parser.parse(response)
     38 print('\n response:\n',parsed_response)
     39 print('-'*100)

File ~/.local/lib/python3.10/site-packages/langchain/output_parsers/structured.py:96, in StructuredOutputParser.parse(self, text)
     94 def parse(self, text: str) -> Any:
     95     expected_keys = [rs.name for rs in self.response_schemas]
---> 96     return parse_and_check_json_markdown(text, expected_keys)

File ~/.local/lib/python3.10/site-packages/langchain/output_parsers/json.py:165, in parse_and_check_json_markdown(text, expected_keys)
    163     json_obj = parse_json_markdown(text)
    164 except json.JSONDecodeError as e:
--> 165     raise OutputParserException(f"Got invalid JSON object. Error: {e}")
    166 for key in expected_keys:
    167     if key not in json_obj:

OutputParserException: Got invalid JSON object. Error: Invalid control character at: line 4 column 27 (char 239)

In [39]:
response_str="""
```json{  "Question": "Given a complex mathematical specification in LaTeX format, synthesize a highly optimized and parallelized C++ code that implements a custom numerical integration algorithm using Gaussian quadrature with adaptive refinement and error estimation.",  "Solution": "```cpp#include <iostream>#include <vector>#include <cmath>#include <thread>#include <mutex>#include <atomic>#include <iomanip>class GaussianQuadrature {public:    GaussianQuadrature(int order) : order(order), weights(order + 1), points(order + 1) {        // Initialize weights and points for Gaussian quadrature        // ...    }    double integrate(std::function<double(double)> f, double a, double b, double tolerance = 1e-6) {        double result, error;        refine(f, a, b, result, error, tolerance);        return result;    }private:    int order;    std::vector<double> weights;    std::vector<double> points;    void refine(std::function<double(double)> f, double a, double b, double& result, double& error, double tolerance) {        double mid = (a + b) / 2.0;        double leftResult, rightResult, leftError, rightError;        std::atomic<bool> done;        std::mutex mtx;        std::thread t1([&, this]() {            refine(f, a, mid, leftResult, leftError, tolerance / 2.0);            done = true;        });        std::thread t2([&, this]() {            if (!done) {                refine(f, mid, b, rightResult, rightError, tolerance / 2.0);            }        });        t1.join();        t2.join();        error = std::max(leftError, rightError);        if (error < tolerance) {            result = (leftResult + rightResult) + (b - a) * weights[order] * f(mid);        } else {            refine(f, a, mid, result, error, tolerance);        }    }};int main() {    GaussianQuadrature quad(10);    auto function = [](double x) { return std::exp(-x * x); };    double integral = quad.integrate(function, 0.0, 1.0);    std::cout << "Numerical integral: " << std::setprecision(15) << integral << std::endl;    return 0;}```",  "Concept": "Code synthesis and parallelization"}```The solution provided is a C++ class implementing a Gaussian quadrature-based numerical integration algorithm with adaptive refinement. The `refine` method uses parallel threads to divide the interval and estimate the integral and error in parallel, ensuring efficient computation. The problem is complex due to the combination of mathematical specification, adaptive refinement, error estimation, and parallelization.
"""
data=json.loads(response_str,strict=False)
print(data)

JSONDecodeError: Expecting value: line 2 column 1 (char 1)

In [15]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"Question": string  // Question generated by the model based on the instruction
	"Solution": string  // Solution generated by the model for its generated question
	"Concept": string  // Concept mentioned in generated the question
}
```


### Testing outlines to get JSON results

In [11]:
from openai import AsyncOpenAI
import tiktoken
from outlines.models.openai import OpenAI, OpenAIConfig
from pydantic import BaseModel
from outlines import models
from outlines import generate
from langchain.output_parsers import StructuredOutputParser,ResponseSchema

response_schemas = [
    ResponseSchema(name="Question", description="Question generated by the model based on the instruction"),
    ResponseSchema(name="Solution", description="Solution generated by the model for its generated question"),
    ResponseSchema(name="Concept", description="Concept mentioned in generated the question"),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

levels=["medium","extremely difficult"]
prompt_template="""Generate a question and its response with example code in suitable language for Area: “{L2_TASK}”, Sub Area: “{L3_TASK}”. The level of complexity has to be {LEVEL}.
{format_instructions}
"""
prompt=PromptTemplate(
    template=prompt_template,
    input_variables=["L2_TASK","L3_TASK","LEVEL","format_instructions"],
    output_parser=output_parser
    )

schema = """
{
  "title": "Questions",
  "type": "object",
  "properties": {
    "Question": {"type": "string"},
    "Solution": {"type": "string"},
    "Concept": {"type": "string"}
  }
}
"""


def query_qwen_outline(query_str,system_content,schema):
    client = openai.OpenAI(api_key = openai.api_key, base_url=openai.api_base)
    config = OpenAIConfig(model="Qwen/Qwen1.5-72B-Chat",temperature=0.7,max_tokens=4096)
    #client = AsyncOpenAI()
    #tokenizer = tiktoken.encoding_for_model("gpt-4")
    model = OpenAI(client,config)
    generator = generate.json(model, schema)
    json_response = generator(query_str)
    return json_response



In [12]:
level="extremely difficult"
l2_task="Code completion"
l3_task="Complete a function"
sys_msg="You are a helpful assistant"
input_prompt = prompt.format_prompt(L2_TASK=l2_task,L3_TASK=l3_task,LEVEL=level,format_instructions=format_instructions).to_string()
out=query_qwen_outline(input_prompt,sys_msg,schema)
print(out)

NotImplementedError: Cannot use JSON Schema-structure generation with an OpenAI model due to the limitations of the OpenAI API